# Weighted Rating Based Recommender

In [1]:
import pandas as pd

## Connecting to the database

In [2]:
%run db_connection.ipynb

Connecting with connection string : postgresql://postgres:letmein@db:5432/recommenderdb
 * postgresql://postgres:***@db:5432/recommenderdb
1 rows affected.
 * postgresql://postgres:***@db:5432/recommenderdb
1 rows affected.
 * postgresql://postgres:***@db:5432/recommenderdb
3 rows affected.


In [3]:
%%sql movies_metadata <<

SELECT * FROM recommender.movies_metadata;

 * postgresql://postgres:***@db:5432/recommenderdb
9989 rows affected.
Returning data to local variable movies_metadata


In [4]:
movies_metadata.sample()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
485,False,None,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,59930.0,tt0110137,en,The Inkwell,The Inkwell is about a 16-year-old boy coming ...,...,1994-04-22,"$8,880,705.00",110.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Summer's Never Been So Much Fun!,The Inkwell,False,3.9,5.0


In [8]:
movies_metadata.shape

(9989, 24)

In [5]:
#Calculate the number of votes garnered by the 80th percentile movie
m = movies_metadata['vote_count'].quantile(0.80)

150.0

In [6]:
m

150.0

In [12]:
#Only consider movies longer than 45 minutes and shorter than 300 minutes
q_movies = movies_metadata[(movies_metadata['runtime'] >= 45) & (movies_metadata['runtime'] <= 300)]

#Only consider movies that have garnered more than m votes
q_movies = q_movies[q_movies['vote_count'] >= m]

#Inspect the number of movies that made the cut
q_movies.shape

(1994, 24)

In [13]:
# Calculate C
C = movies_metadata['vote_average'].mean()


In [14]:
C

6.12011212333568

In [15]:
# Function to compute the IMDB weighted rating for each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Compute the weighted score
    return (v/(v+m) * R) + (m/(m+v) * C)

In [16]:
# Compute the score using the weighted_rating function defined above
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [17]:
#Sort movies in descending order of their scores
q_movies = q_movies.sort_values('score', ascending=False)

In [18]:
q_movies[['title', 'vote_count', 'vote_average', 'score', 'runtime']].head(6)

,title,vote_count,vote_average,score,runtime
328,The Shawshank Redemption,8358.0,8.5,8.458041,142.0
842,The Godfather,6024.0,8.5,8.442180,175.0
2859,Fight Club,9678.0,8.3,8.266729,139.0
304,Pulp Fiction,8670.0,8.3,8.262927,154.0
538,Schindler's List,4436.0,8.3,8.228700,195.0
5487,Spirited Away,3968.0,8.3,8.220597,125.0
